# 리콜포탈 크롤링


In [33]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import chromedriver_autoinstaller
import pandas as pd
from selenium.webdriver.common.by import By

# 크롬 드라이버 자동 설치 및 설정
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

# 첫 페이지 URL 설정
url = "https://www.car.go.kr/ri/stat/list.do"

# 첫 페이지 로드
driver.get(url)
time.sleep(3)  # 페이지 로딩을 기다림

def get_page_data(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    items = soup.select('ul.board-hrznt-list > li')
    data = []

    for item in items:
        title = item.select_one('strong').get_text(strip=True)
        ministry = item.select_one('ol > li').get_text(strip=True)
        date = item.select('ol > li')[1].get_text(strip=True)  # 두 번째 <li> 태그에서 날짜 추출
        data.append({'Title': title, 'Ministry': ministry, 'Date': date})
    
    return data

all_data = []

# 40페이지까지 크롤링 (예: 10단위 페이지를 고려)
for page in range(1, 571):
    page_data = get_page_data(driver)
    all_data.extend(page_data)
    
    if page < 570:
        try:
            # 현재 페이지가 10의 배수일 경우 "다음페이지" 클릭
            if page % 10 == 0:
                next_page = driver.find_element(By.XPATH, "//a[@title='다음페이지']")
            else:
                next_page = driver.find_element(By.XPATH, f"//a[contains(text(),'{page + 1}')]")
            
            next_page.click()
            time.sleep(1)  # 페이지 로딩을 기다림
        except Exception as e:
            print(f"페이지 {page + 1}로 이동 실패: {e}")
            break

# 크롤링 완료 후 브라우저 종료
driver.quit()

# 데이터를 DataFrame으로 변환
df = pd.DataFrame(all_data)

# DataFrame 출력
df



                                                  Title Ministry        Date
0     [포드] Navigator 등 2차종 - 취급 설명서(사용자 매뉴얼 - 3열 가운데...    국토교통부  2024-08-02
1                 [야마하] LTS125(악시스) - 속도 케이블(계기판) 관련 리콜    국토교통부  2024-08-02
2                       [만트럭] TGS 카고 등 5차종 - 시가소켓 관련 리콜    국토교통부  2024-08-01
3                                [이비온] E6 - 승강구높이 관련 리콜    국토교통부  2024-08-01
4                                 [이비온] E6 - 승객좌석 관련 리콜    국토교통부  2024-08-01
...                                                 ...      ...         ...
2845                      [메르세데스-벤츠] C, CLS. E-Class 리콜    국토교통부  2015-02-16
2846                                      [한국지엠] 카마로 리콜    국토교통부  2015-02-13
2847                                   [포드코리아] 이스케이프 리콜    국토교통부  2015-02-06
2848                              [기흥모터스] FXDF 외 9차종 리콜    국토교통부  2015-02-02
2849                             [BMW코리아] 328i 등 4차종 리콜    국토교통부  2015-01-30

[2850 rows x 3 columns]


In [35]:
df.to_excel('리콜.xlsx')

In [36]:
df

,Title,Ministry,Date
0,[포드] Navigator 등 2차종 - 취급 설명서(사용자 매뉴얼 - 3열 가운데...,국토교통부,2024-08-02
1,[야마하] LTS125(악시스) - 속도 케이블(계기판) 관련 리콜,국토교통부,2024-08-02
2,[만트럭] TGS 카고 등 5차종 - 시가소켓 관련 리콜,국토교통부,2024-08-01
3,[이비온] E6 - 승강구높이 관련 리콜,국토교통부,2024-08-01
4,[이비온] E6 - 승객좌석 관련 리콜,국토교통부,2024-08-01
...,...,...,...
2845,"[메르세데스-벤츠] C, CLS. E-Class 리콜",국토교통부,2015-02-16
2846,[한국지엠] 카마로 리콜,국토교통부,2015-02-13
2847,[포드코리아] 이스케이프 리콜,국토교통부,2015-02-06
2848,[기흥모터스] FXDF 외 9차종 리콜,국토교통부,2015-02-02


In [30]:


# title 컬럼에서 [회사] 차종 - 문제 내용 분리
def split_title(title):
    try:
        # 회사 추출
        if ']' in title:
            company = title.split(']')[0].replace('[', '').strip()
            # 나머지 부분 추출
            remaining = title.split(']')[1].strip()
        else:
            # [회사] 형식이 아닌 경우 처리
            company = title.split(' ')[0].strip()
            remaining = ' '.join(title.split(' ')[1:]).strip()
        
        # 차종과 문제 내용 분리
        if '-' in remaining:
            model, issue = remaining.split('-', 1)
        else:
            model, issue = remaining, ""
        
        model = model.strip()
        issue = issue.strip()
    
    except Exception as e:
        company, model, issue = "Unknown", "Unknown", title.strip()
    
    return pd.Series([company, model, issue])
df=pd.read_excel('/Users/admin/softeer/project/리콜.xlsx')
# 새로운 컬럼 생성
df[['Company', 'Model', 'Issue']] = df['Title'].apply(split_title)

# 기존 title 컬럼 제거 (선택 사항)
df = df.drop(columns=['Title'])

# 정리된 데이터 출력
print("정리된 데이터:")
print(df.head())

정리된 데이터:
   Unnamed: 0 Ministry        Date Company            Model  \
0           0    국토교통부  2024-08-02      포드  Navigator 등 2차종   
1           1    국토교통부  2024-08-02     야마하      LTS125(악시스)   
2           2    국토교통부  2024-08-01     만트럭     TGS 카고 등 5차종   
3           3    국토교통부  2024-08-01     이비온               E6   
4           4    국토교통부  2024-08-01     이비온               E6   

                                               Issue  
0  취급 설명서(사용자 매뉴얼 - 3열 가운데 좌석 머리 받침대 조작 및 제거 방법에 ...  
1                                  속도 케이블(계기판) 관련 리콜  
2                                         시가소켓 관련 리콜  
3                                        승강구높이 관련 리콜  
4                                         승객좌석 관련 리콜  


In [33]:
df.to_excel('/Users/admin/softeer/project/리콜2.xlsx')